In [1]:
!which python; python -V;

# This makes the diagrams to more reliably appear in Jupyter environment
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# This will cause the ephemerides to be imported from JPL horizons system
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


<ScienceState solar_system_ephemeris: 'jpl'>

In [2]:
# We will use the interplanetary package from perylune to calculate interplanetary transfers
from perylune.interplanetary import *

In [3]:

# These are the Hohmann trajectiories we want to investigate.
trajectories = [
    # PLANETS:
    ["earth", "mercury"],
    ["earth", "venus"],
    ["earth", "mars"],
    ["earth", "jupiter"],
    ["earth", "saturn"],
    ["earth", "uranus"],
    ["earth", "neptune"],

    # DWARF PLANETS
    ["earth", "ceres"],
    ["earth", "eris"],
    ["earth", "pluto"],
    ["earth", "makemake"],
    ["earth", "haumea"],

    # ASEROIDS
    ["earth", "vesta"],
    ["earth", "bernievolz"]
]

# Print the header
print(transfer_vel_header())

# And now calculate the trajectories
for traj in trajectories:
    v = transfer_vel(traj[0], traj[1], None)
    txt = transfer_vel_format(traj[0], traj[1], v, ",")
    print(txt)

# all values in km/s, except noted otherwise
#departure, arrival, heliocentric vel @ departure, heliocentric vel @ arrival, hohmann dep burn, hohmann arrival burn, time of flight [days], time of flight [years], Hohman1 delta-v, Hohmann2 delta-v
earth, mercury, 29.79, 47.87, 22.25, 57.48, 105.47, 0.289, 7.532, 9.610
earth, venus, 29.79, 35.02, 27.29, 37.73, 146.06, 0.400, 2.494, 2.705
earth, mars, 29.79, 24.13, 32.73, 21.48, 258.83, 0.709, 2.946, 2.650
earth, jupiter, 29.79, 13.06, 38.58, 7.41, 997.71, 2.732, 8.794, 5.644
earth, saturn, 29.79, 9.62, 40.09, 4.18, 2221.95, 6.083, 10.298, 5.439
earth, uranus, 29.79, 6.80, 41.07, 2.14, 5864.06, 16.055, 11.283, 4.658
earth, neptune, 29.79, 5.42, 41.45, 1.37, 11274.23, 30.867, 11.659, 4.046
earth, ceres, 29.79, 17.91, 36.10, 13.05, 471.92, 1.292, 6.316, 4.859
earth, eris, 29.79, 3.61, 41.82, 0.62, 36947.08, 101.156, 12.031, 2.998
earth, pluto, 29.79, 4.74, 41.60, 1.05, 16600.16, 45.449, 11.814, 3.688
earth, makemake, 29.79, 4.42, 41.67, 0.92,